## Day 9

### Part 1

With your neighbor happily enjoying their video game, you turn your attention to an open data port on the little screen in the seat in front of you.

Though the port is non-standard, you manage to connect it to your computer through the clever use of several paperclips. Upon connection, the port outputs a series of numbers (your puzzle input).

The data appears to be encrypted with the eXchange-Masking Addition System (XMAS) which, conveniently for you, is an old cypher with an important weakness.

XMAS starts by transmitting a preamble of 25 numbers. After that, each number you receive should be the sum of any two of the 25 immediately previous numbers. The two numbers will have different values, and there might be more than one such pair.

For example, suppose your preamble consists of the numbers 1 through 25 in a random order. To be valid, the next number must be the sum of two of those numbers:

- 26 would be a valid next number, as it could be 1 plus 25 (or many other pairs, like 2 and 24).
- 49 would be a valid next number, as it is the sum of 24 and 25.
- 100 would not be valid; no two of the previous 25 numbers sum to 100.
- 50 would also not be valid; although 25 appears in the previous 25 numbers, the two numbers in the pair must be different.

Suppose the 26th number is 45, and the first number (no longer an option, as it is more than 25 numbers ago) was 20. Now, for the next number to be valid, there needs to be some pair of numbers among 1-19, 21-25, or 45 that add up to it:
- 26 would still be a valid next number, as 1 and 25 are still within the previous 25 numbers.
- 65 would not be valid, as no two of the available numbers sum to it.
- 64 and 66 would both be valid, as they are the result of 19+45 and 21+45 respectively.

Here is a larger example which only considers the previous 5 numbers (and has a preamble of length 5):

- 35
- 20
- 15
- 25
- 47
- 40
- 62
- 55
-  65
- 95
- 102
- 117
- 150
- 182
- 127
- 219
- 299
- 277
- 309
- 576

In this example, after the 5-number preamble, almost every number is the sum of two of the previous 5 numbers; the only number that does not follow this rule is 127.

The first step of attacking the weakness in the XMAS data is to find the first number in the list (after the preamble) which is not the sum of two of the 25 numbers before it. What is the first number that does not have this property?

In [1]:
# import packages
import pandas as pd
import numpy as np

In [2]:
numbers = pd.read_csv('input_data/Day9.txt', header=None)

numbers.shape

numbers.columns=['number']

numbers.head()

,number
0,16
1,45
2,42
3,47
4,31


In [3]:
def valid_num(num, list_of_nums):
    '''takes a number and list of numbers and returns
    True if the number is the sum of any of the numbers in the list
    False if the number is not the summ of any of the numbers in the list'''
    
    for idx, val1 in enumerate(list_of_nums):
        for val2 in list_of_nums[idx+1:]:
            if val1+val2 == num:
                return True
    return False

In [4]:
# Let's test this
test_num=40
test_list = [35,20,15,25,47,40]
valid_num(test_num,test_list)

True

In [5]:
# Now let's loop through our dataframe and do this based on a variable length list of previous numbers

# This is the length of the list we're going to test and 
# also is our starting position
length_of_list = 25
position = length_of_list

total_numbers = numbers.shape[0]

# Now we loop through our list of numbers, grabbing a number and the list of previous numbers
nums_valid = True
while nums_valid and position<total_numbers:
    test_num = numbers.iloc[position]['number']
    prev_list = numbers.iloc[position-length_of_list:position]['number'].to_list()
    
    nums_valid = valid_num(test_num,prev_list)

#    print(position, test_num, prev_list, nums_valid)

    # Increment our position
    position += 1

print('And our bad number is: ', test_num)

And our bad number is:  2089807806


### Part 2
The final step in breaking the XMAS encryption relies on the invalid number you just found: you must find a contiguous set of at least two numbers in your list which sum to the invalid number from step 1.

Again consider the above example:

- 35
- 20
- 15
- 25
- 47
- 40
- 62
- 55
- 65
- 95
- 102
- 117
- 150
- 182
- 127
- 219
- 299
- 277
- 309
- 576

In this list, adding up all of the numbers from 15 through 40 produces the invalid number from step 1, 127. (Of course, the contiguous set of numbers in your actual list might be much longer.)

To find the encryption weakness, add together the smallest and largest number in this contiguous range; in this example, these are 15 and 47, producing 62.

What is the encryption weakness in your XMAS-encrypted list of numbers?

In [6]:
# let's save our invalid number and drop it from our data
invalid_num = test_num
numbers = numbers[numbers['number'] != invalid_num]
numbers.reset_index(inplace=True, drop = True)

In [7]:
numbers.shape

(999, 1)

In [8]:
def find_range(list_of_nums, target):
    '''takes a list of numbers and determines if there is a contiguous range in that list
    which sum to the target number.'''
    
    too_big = False
    pos = 0
    while not too_big:
        
#        print(pos, sum(list_of_nums[0:pos+2]))
        
        if sum(list_of_nums[0:pos+2]) == target:
               return True
        elif sum(list_of_nums[0:pos+2]) > target:
            too_big = True
        
        pos += 1
                 
    return False

In [9]:
# Now we loop through our dataframe, sending in a new list of numbers each time

found_range = False
total_numbers = numbers.shape[0]
start_position = 0

while not found_range:
    
    list_to_check = numbers.iloc[start_position:total_numbers]['number'].tolist()
    start_position += 1
    
    found_range = find_range(list_to_check, invalid_num)

# Ok we've got our list that contains the contiguous range so we just need to go through it
# and find the min and max of the range and sum them.  

max_pos=2
found = False

while not found:
    found = (sum(list_to_check[0:max_pos+2]) == invalid_num)
    max_pos += 1

print(invalid_num, sum(list_to_check[0:max_pos+1]), min((list_to_check[0:max_pos+1])) + max((list_to_check[0:max_pos+1])))

2089807806 2089807806 245848639
